In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import random
from sklearn.preprocessing import StandardScaler


In [190]:
final_df = pd.read_csv('../data/final_plot_emotion_20_genres.csv')
final_df 

,Name,Countries,Genres,Budget(USD)_Inflated,Domestic(USD)_Inflated,Domestic_Percentage,Foreign(USD)_Inflated,Foreign_Percentage,Worldwide(USD)_Inflated,Runtime(mins),...,animation,teen,film adaptation,musical,history,coming of age,sports,black comedy,war film,Worldwide_profit
0,10 cloverfield lane,united states of america,"drama, horror, sci-fi, thriller",6.076746e+06,8.760602e+07,0.654010,4.634613e+07,0.345990,1.339521e+08,103,...,0,0,0,0,0,0,0,0,0,22.043400
1,"10,000 bc",united states of america,"thriller, adventure, costume drama, action, ad...",1.441563e+08,1.301309e+08,0.351333,2.402606e+08,0.648667,3.703914e+08,109,...,0,0,0,0,0,0,0,0,0,2.569373
2,12 rounds,united states of america,"action thrillers, thriller, action, crime fiction",2.726275e+07,1.667757e+07,0.708013,6.877890e+06,0.291987,2.355546e+07,108,...,0,0,0,0,0,0,0,0,0,0.864016
3,12 strong,united states of america,"action, drama, history, war",3.979050e+07,5.209113e+07,0.644274,2.876133e+07,0.355726,8.085246e+07,130,...,0,0,0,0,1,0,0,0,0,2.031954
4,12 years a slave,united states of america,"biography, drama, history",2.539260e+07,7.195247e+07,0.301875,1.663993e+08,0.698125,2.383517e+08,134,...,0,0,0,0,1,0,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,zoolander,united states of america,"parody, comedy of errors, comedy",4.851315e+07,7.826600e+07,0.743197,2.704388e+07,0.256803,1.053099e+08,90,...,0,0,0,0,0,0,0,0,0,2.170749
2685,zoolander 2,united states of america,"action, adventure, comedy, mystery, romance",6.076746e+07,3.506124e+07,0.508592,3.387664e+07,0.491408,6.893788e+07,101,...,0,0,0,0,0,0,0,0,0,1.134454
2686,zoom,united states of america,"science fiction, action, fantasy, adventure",5.014200e+07,1.717625e+07,0.958658,7.407176e+05,0.041342,1.791697e+07,93,...,0,0,0,0,0,0,0,0,0,0.357325
2687,zootopia,united states of america,"action, adventure, animation, comedy, crime, f...",1.823024e+08,4.147601e+08,0.333340,8.294952e+08,0.666660,1.244255e+09,108,...,1,0,0,0,0,0,0,0,0,6.825228


In [191]:
final_df.columns

Index(['Name', 'Countries', 'Genres', 'Budget(USD)_Inflated',
       'Domestic(USD)_Inflated', 'Domestic_Percentage',
       'Foreign(USD)_Inflated', 'Foreign_Percentage',
       'Worldwide(USD)_Inflated', 'Runtime(mins)', 'Rating', 'Audience_Score',
       'Critics_Score', 'plot_summary', 'Month', 'Day', 'Year', 'emotion',
       'emotion_score', 'drama', 'comedy', 'thriller', 'action', 'adventure',
       'romance film', 'horror', 'fantasy', 'mystery', 'crime fiction',
       'indie', 'biography', 'family film', 'sci-fi', 'period piece',
       'animation', 'teen', 'film adaptation', 'musical', 'history',
       'coming of age', 'sports', 'black comedy', 'war film',
       'Worldwide_profit'],
      dtype='object')

## preparing the data

In [192]:
# Dropping columns deemed irrelevant for the model
df_ols = final_df.drop(columns=["Genres", "Day","Name", "plot_summary", "Countries", "Worldwide(USD)_Inflated", "Domestic(USD)_Inflated", "Foreign(USD)_Inflated", "Worldwide(USD)_Inflated"])

In [193]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,animation,teen,film adaptation,musical,history,coming of age,sports,black comedy,war film,Worldwide_profit
0,6.076746e+06,0.654010,0.345990,103,pg-13,0.79,0.91,Mar,2016,fear,...,0,0,0,0,0,0,0,0,0,22.043400
1,1.441563e+08,0.351333,0.648667,109,pg-13,0.37,0.10,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,2.569373
2,2.726275e+07,0.708013,0.291987,108,pg-13,0.45,0.31,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0.864016
3,3.979050e+07,0.644274,0.355726,130,r,0.62,0.50,Jan,2018,fear,...,0,0,0,0,1,0,0,0,0,2.031954
4,2.539260e+07,0.301875,0.698125,134,r,0.90,0.95,Oct,2013,sadness,...,0,0,0,0,1,0,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,4.851315e+07,0.743197,0.256803,90,pg-13,0.80,0.64,Sep,2001,anger,...,0,0,0,0,0,0,0,0,0,2.170749
2685,6.076746e+07,0.508592,0.491408,101,pg-13,0.20,0.22,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,1.134454
2686,5.014200e+07,0.958658,0.041342,93,pg,0.34,0.05,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,0,0.357325
2687,1.823024e+08,0.333340,0.666660,108,pg,0.92,0.98,Mar,2016,neutral,...,1,0,0,0,0,0,0,0,0,6.825228


In [194]:
# Standardizing 

columns_to_scale = ['Budget(USD)_Inflated', 'Domestic_Percentage', 'Foreign_Percentage', 'Runtime(mins)', 'Audience_Score', 'Critics_Score', 'emotion_score']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_ols[columns_to_scale])
df_ols[columns_to_scale] = scaled_data

df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,animation,teen,film adaptation,musical,history,coming of age,sports,black comedy,war film,Worldwide_profit
0,-0.820351,0.391747,-0.391747,-0.291947,pg-13,1.012762,1.412014,Mar,2016,fear,...,0,0,0,0,0,0,0,0,0,22.043400
1,1.305928,-0.877514,0.877514,0.052120,pg-13,-1.230673,-1.539543,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,2.569373
2,-0.494109,0.618207,-0.618207,-0.005225,pg-13,-0.803352,-0.774325,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0.864016
3,-0.301195,0.350920,-0.350920,1.256351,r,0.104705,-0.081984,Jan,2018,fear,...,0,0,0,0,1,0,0,0,0,2.031954
4,-0.522908,-1.084916,1.084916,1.485729,r,1.600328,1.557770,Oct,2013,sadness,...,0,0,0,0,1,0,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,-0.166876,0.765751,-0.765751,-1.037423,pg-13,1.066177,0.428161,Sep,2001,anger,...,0,0,0,0,0,0,0,0,0,2.170749
2685,0.021828,-0.218058,0.218058,-0.406635,pg-13,-2.138730,-1.102276,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,1.134454
2686,-0.141793,1.669279,-1.669279,-0.865390,pg,-1.390919,-1.721738,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,0,0.357325
2687,1.893336,-0.952969,0.952969,-0.005225,pg,1.707158,1.667086,Mar,2016,neutral,...,1,0,0,0,0,0,0,0,0,6.825228


In [195]:
""""
# This part should already be in the main anaylsis!
# Filtering genres to only save top 20

# Assuming `final_df` is already loaded

# Split the genres into a list and explode the DataFrame
genres_exploded = final_df['Genres'].str.split(',').explode().str.strip()

# Count the occurrences of each genre
top_genres = genres_exploded.value_counts().head(20).index

# Filter the original DataFrame to only keep rows containing the top 20 genres
final_df['Filtered_Genres'] = final_df['Genres'].apply(
    lambda x: ', '.join([genre for genre in x.split(',') if genre.strip() in top_genres])
)

# Drop rows with no genres in the top 20
final_df = final_df[final_df['Filtered_Genres'].str.strip() != '']

# Optional: Drop the original Genres column and rename Filtered_Genres to Genres
final_df.drop(columns=['Genres'], inplace=True)
final_df.rename(columns={'Filtered_Genres': 'Genres'}, inplace=True)

# Print the updated DataFrame
final_df

-------------------------------------------------------------

# Splitting and one-hot encoding genres
genres_split = final_df['Genres'].str.get_dummies(sep=',')

# Combining one-hot encoded genres with final_df
df_ols = pd.concat([final_df, genres_split], axis=1)
df_ols = df_ols.drop(columns=['Genres'])


df_ols
"""

'"\n# This part should already be in the main anaylsis!\n# Filtering genres to only save top 20\n\n# Assuming `final_df` is already loaded\n\n# Split the genres into a list and explode the DataFrame\ngenres_exploded = final_df[\'Genres\'].str.split(\',\').explode().str.strip()\n\n# Count the occurrences of each genre\ntop_genres = genres_exploded.value_counts().head(20).index\n\n# Filter the original DataFrame to only keep rows containing the top 20 genres\nfinal_df[\'Filtered_Genres\'] = final_df[\'Genres\'].apply(\n    lambda x: \', \'.join([genre for genre in x.split(\',\') if genre.strip() in top_genres])\n)\n\n# Drop rows with no genres in the top 20\nfinal_df = final_df[final_df[\'Filtered_Genres\'].str.strip() != \'\']\n\n# Optional: Drop the original Genres column and rename Filtered_Genres to Genres\nfinal_df.drop(columns=[\'Genres\'], inplace=True)\nfinal_df.rename(columns={\'Filtered_Genres\': \'Genres\'}, inplace=True)\n\n# Print the updated DataFrame\nfinal_df\n\n---------

In [196]:
# one-hot encoding rating column, dropping the first column in order to avoid introducing multicollinearity
df_ols = pd.get_dummies(df_ols, columns=['Rating'], drop_first=True)
df_ols = df_ols.drop(columns="Rating_nc-17") # Only 5 nc-17 movies, not considered for analysis
# one-hot encoding month column, dropping first column for same reason
df_ols = pd.get_dummies(df_ols, columns=['Month'], drop_first=True)
# same thing for the emotion column
df_ols = pd.get_dummies(df_ols, columns=['emotion'], drop_first=True)


In [197]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Audience_Score,Critics_Score,Year,emotion_score,drama,comedy,...,Month_May,Month_Nov,Month_Oct,Month_Sep,emotion_disgust,emotion_fear,emotion_joy,emotion_neutral,emotion_sadness,emotion_surprise
0,-0.820351,0.391747,-0.391747,-0.291947,1.012762,1.412014,2016,0.050657,1,0,...,False,False,False,False,False,True,False,False,False,False
1,1.305928,-0.877514,0.877514,0.052120,-1.230673,-1.539543,2008,0.504088,1,0,...,False,False,False,False,False,True,False,False,False,False
2,-0.494109,0.618207,-0.618207,-0.005225,-0.803352,-0.774325,2009,0.056938,0,0,...,False,False,False,False,False,False,False,False,False,False
3,-0.301195,0.350920,-0.350920,1.256351,0.104705,-0.081984,2018,1.226752,1,0,...,False,False,False,False,False,True,False,False,False,False
4,-0.522908,-1.084916,1.084916,1.485729,1.600328,1.557770,2013,0.964755,1,0,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,-0.166876,0.765751,-0.765751,-1.037423,1.066177,0.428161,2001,-0.726523,0,1,...,False,False,False,True,False,False,False,False,False,False
2685,0.021828,-0.218058,0.218058,-0.406635,-2.138730,-1.102276,2016,1.495326,0,1,...,False,False,False,False,False,False,False,False,False,True
2686,-0.141793,1.669279,-1.669279,-0.865390,-1.390919,-1.721738,2006,0.055972,0,0,...,False,False,False,False,False,False,False,True,False,False
2687,1.893336,-0.952969,0.952969,-0.005225,1.707158,1.667086,2016,0.256964,0,1,...,False,False,False,False,False,False,False,True,False,False


## Defining features and labels for the 4 different models:
Labels: Foreign/domestic percentage for prediction
Predictors: before and after 2010

In [198]:
# Movies before 2010
df_before_2010 = df_ols[df_ols['Year'] < 2010]

# Movies including and after 2010
df_2010_and_after = df_ols[df_ols['Year'] >= 2010]

### 1st model: Foreign Percentage prediction for years before 2010 (have to seperate years)

In [199]:
# Defining features and labels
y = df_ols["Foreign_Percentage"]
X = df_ols.drop(columns= ["Foreign_Percentage", 'Domestic_Percentage'])

# Adding a constant column for the OLS model
X = sm.add_constant(X)

feature_names =  list(X.columns)
label_name = "Foreign_Percentage"

# Training the OLS model
X = pd.DataFrame(X, columns=feature_names, dtype=float)  # Ensure predictors are numeric
y = pd.Series(y, name=label_name, dtype=float) # To include label name
ols_model = sm.OLS(y, X).fit()

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     19.15
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          5.21e-143
Time:                        23:49:09   Log-Likelihood:                -3391.9
No. Observations:                2689   AIC:                             6888.
Df Residuals:                    2637   BIC:                             7194.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  -61.6763 

### "2nd" model: Foreign Percentage prediction for years before 2010 (have to seperate years)

In [202]:
# Defining features and labels
y = df_ols["Domestic_Percentage"]
X = df_ols.drop(columns= ["Foreign_Percentage", 'Domestic_Percentage'])

# Adding a constant column for the OLS model
X = sm.add_constant(X)

feature_names =  list(X.columns)
label_name = "Foreign_Percentage"

# Training the OLS model
X = pd.DataFrame(X, columns=feature_names, dtype=float)  # Ensure predictors are numeric
y = pd.Series(y, name=label_name, dtype=float) # To include label name
ols_model = sm.OLS(y, X).fit()

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     19.15
Date:                Thu, 19 Dec 2024   Prob (F-statistic):          5.21e-143
Time:                        23:49:32   Log-Likelihood:                -3391.9
No. Observations:                2689   AIC:                             6888.
Df Residuals:                    2637   BIC:                             7194.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   61.6763 

### Before 2010 with foreign

In [203]:
# Defining features and labels
y = df_before_2010["Foreign_Percentage"]
X = df_before_2010.drop(columns= ["Foreign_Percentage", 'Domestic_Percentage'])

# Adding a constant column for the OLS model
X = sm.add_constant(X)

feature_names =  list(X.columns)
label_name = "Foreign_Percentage"

# Training the OLS model
X = pd.DataFrame(X, columns=feature_names, dtype=float)  # Ensure predictors are numeric
y = pd.Series(y, name=label_name, dtype=float) # To include label name
ols_model = sm.OLS(y, X).fit()

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     9.310
Date:                Thu, 19 Dec 2024   Prob (F-statistic):           5.43e-57
Time:                        23:51:21   Log-Likelihood:                -1459.9
No. Observations:                1232   AIC:                             3024.
Df Residuals:                    1180   BIC:                             3290.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 -106.8019 

### Including and after 2010, foreign prediction

In [204]:
# Defining features and labels
y = df_2010_and_after["Foreign_Percentage"]
X = df_2010_and_after.drop(columns= ["Foreign_Percentage", 'Domestic_Percentage'])

# Adding a constant column for the OLS model
X = sm.add_constant(X)

feature_names =  list(X.columns)
label_name = "Foreign_Percentage"

# Training the OLS model
X = pd.DataFrame(X, columns=feature_names, dtype=float)  # Ensure predictors are numeric
y = pd.Series(y, name=label_name, dtype=float) # To include label name
ols_model = sm.OLS(y, X).fit()

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     10.30
Date:                Thu, 19 Dec 2024   Prob (F-statistic):           5.26e-66
Time:                        23:52:49   Log-Likelihood:                -1866.8
No. Observations:                1457   AIC:                             3838.
Df Residuals:                    1405   BIC:                             4112.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    7.6724 